This notebook is based on a tutorial prepared by Dr. Brinnae Bent 

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import random
import numpy as np